In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
# import the path
import os
path = os.path.abspath(os.path.join("..", "08_portfolio_management", "portfolio_example.xlsx"))

# read the Excel file
df = pd.read_excel(path, sheet_name=None)

# show the first sheet's data
first_sheet_name = list(df.keys())[0]

# dict to dataframe
df = pd.DataFrame(df[first_sheet_name])

# df

In [3]:
# add here Last Closing Price data download from yf or tv
# for tv, need to set the correct exchange/market

"""
Looks like Yahoo change their API regularly.
I have to use fast_info instead of info and then there's a key called 'lastPrice':

CurrentPrice = Commod.fast_info['lastPrice']
""";
# import yfinance as yf
# from tvDatafeed import TvDatafeed, Interval
# tv = TvDatafeed()

In [4]:
# rename columns for clarity
df.rename(columns={'Last Closing Price': 'Last_Price', 'Somma di Q.ty': 'Quantity'}, inplace=True)

# calculate portfolio value for each position
df['Position_Value'] = df['Last_Price'] * df['Quantity']

# display the portfolio
# print("Portfolio Overview:")
# df.head()

In [5]:
# print(f"\nTotal Portfolio Value: ${df['Position_Value'].sum():,.2f}")
print(f"Number of positions: {len(df)}")

Number of positions: 50


In [6]:
# portfolio pct composition
df['Position_Weight'] = (df['Position_Value'] / df['Position_Value'].sum())*100
# df.head()

In [7]:
# # download df as csv
# df.to_csv("portfolio_data.csv", index=False)

In [8]:
# top 10 holdings
top_10 = df.nlargest(10, 'Position_Value')
# top_10

In [ ]:
# Position_Weight of top 10
top_10_weight = top_10['Position_Weight'].sum()
# print(f"\nTop 10 Holdings Weight: {top_10_weight:.2f}%")


Top 10 Holdings Weight: 86.66%


### Portfolio Value Update

In [10]:
# Last_Price data download from yfinance
import yfinance as yf
import datetime

def download_last_price_individual(tickers, start_date, end_date):
    """
    Download last prices for each ticker individually to handle failures gracefully
    """
    last_prices = {}
    
    for ticker in tickers:
        try:
            print(f"Downloading data for {ticker}...")
            # Download data for individual ticker
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)
            
            if not data.empty and 'Close' in data.columns:
                # Get the most recent price and ensure it's a scalar value
                last_price = float(data['Close'].iloc[-1])
                last_prices[ticker] = last_price
                print(f" {ticker}: ${last_price:.2f}")
            else:
                print(f" {ticker}: No data available")
                last_prices[ticker] = None
                
        except Exception as e:
            print(f" {ticker}: Error - {str(e)}")
            last_prices[ticker] = None
    
    return last_prices

tickers = df['Ticker'].tolist()

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=10)

print(f"Downloading last prices for {len(tickers)} tickers...")
print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print("-" * 50)

last_prices = download_last_price_individual(tickers, start_date, end_date)

# update Last_Price in df
print("\n" + "-" * 50)
print("Updating portfolio with new prices...")

# Update prices, keeping original price if download failed
for ticker in df['Ticker']:
    if ticker in last_prices and last_prices[ticker] is not None:
        df.loc[df['Ticker'] == ticker, 'Last_Price'] = last_prices[ticker]
    else:
        print(f"  Keeping original price for {ticker} (download failed)")

# recalculate Position_Value and Position_Weight
df['Position_Value'] = df['Last_Price'] * df['Quantity']

df['Position_Weight'] = (df['Position_Value'] / df['Position_Value'].sum())*100

# display updated portfolio
print("\nUpdated Portfolio Overview:")
print(df[['Ticker', 'Last_Price', 'Position_Value', 'Position_Weight']])

# total portfolio value
total_value = df['Position_Value'].sum()
print(f"\nTotal Portfolio Value: ${total_value:,.2f}")

# number of positions
num_positions = len(df)
print(f"Number of positions: {num_positions}")

# Show which tickers were successfully updated
successful_updates = [ticker for ticker, price in last_prices.items() if price is not None]
failed_updates = [ticker for ticker, price in last_prices.items() if price is None]

print(f"\nSuccessfully updated: {len(successful_updates)} tickers")
if successful_updates:
    print(f"  Updated: {', '.join(successful_updates)}")

if failed_updates:
    print(f"\n  Failed to update: {len(failed_updates)} tickers")
    print(f"  Failed: {', '.join(failed_updates)}")

# top 10 holdings
top_10 = df.nlargest(10, 'Position_Value')
print("\nTop 10 Holdings:")
print(top_10[['Ticker', 'Position_Value', 'Position_Weight']])

top_10_weight = top_10['Position_Weight'].sum()
print(f"\nTop 10 Holdings Weight: {top_10_weight:.2f}%")

# save updated portfolio to csv
df.to_csv("portfolio_data.csv", index=False)
print(f"\nPortfolio data saved to 'portfolio_data.csv'")

Date range: 2025-12-08 to 2025-12-18
--------------------------------------------------
 BNB-USD: $846.62
 BR50.MI: $22.91
 BTC-USD: $88191.43
 DOGE-USD: $0.13
 EQQQ.DE: $523.20
 ETH-USD: $2949.29
 EXXT.DE: $208.05
 GOOGL: $302.90
 IEEM.MI: $45.57
 IWDE.MI: $106.64
 LULU: $221.40
 MA: $566.23
 MC.PA: $639.70
 META: $667.85
 MSTR: $165.49
 NKE: $66.54
 PYPL: $59.72
 RACE: $371.68
 TSLA: $488.85
 V: $345.66
 CRM: $258.73
 PATH: $16.14
 SOL-USD: $127.04
 PENGU34466-USD: $0.01
 DOOD-USD: $0.01
 CAKE-USD: $1.87
 CMG: $37.72
 COIN: $249.30
 SOFI: $26.65
 TSCO: $52.32
 FTNT: $79.39
 TMDX: $123.62
 ORLY: $91.69
 LYFT: $19.96
 DOCN: $45.42
 UNH: $328.60
 CHD: $85.73
 TSCO: $52.31
 SHW: $332.54
 BRK-B: $504.45
 IEX: $178.23
 BIDU: $121.97
 CL: $78.00
 DECK: $102.18
 ORCL: $182.01
 TRGP: $180.77
 DUOL: $182.87
 MOH: $163.51
 MDT: $98.86
 BALL: $52.86

--------------------------------------------------
Updating portfolio with new prices...

Updated Portfolio Overview:
            Ticker    Last_Pr